In [30]:
import pandas as pd
import seaborn as sns
import numpy as np
import os.path as op
import glob
import matplotlib.pyplot as plt
import ast
# import moss
import csv
import random
%matplotlib inline
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning
import scipy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tools.sm_exceptions import ConvergenceWarning
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm
from matplotlib.ticker import PercentFormatter
from statsmodels.stats.anova import AnovaRM
from sklearn.preprocessing import StandardScaler
from matplotlib.cm import get_cmap
from plot_utils import combined_rolling_average_and_model_fit, combined_sliding_window_and_model_fit_split_col
from common_functions import remove_unit_variance
from matplotlib.ticker import FuncFormatter, MultipleLocator

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import json
from matplotlib.ticker import FuncFormatter, MultipleLocator

In [31]:
scaler = StandardScaler()

df = pd.read_csv('pilot_total.csv')

/var/folders/h_/pm5z54tn7rj170br8494ljz80000gn/T/ipykernel_22342/3283417062.py:3: DtypeWarning: Columns (254,255,256,257,258,259) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('pilot_total.csv')


In [32]:
df['Accuracy'] = df['resp_correct']

I have a dataframe df, a column 'Accuracy' that has 1 or 0 for correct and incorrect answers, has a column 'participant', similarity columns "Tested - Untested V2 Distractor Similarity" and "Tested - Untested IT Distractor Similarity". The dataframe has all the trials for all participants. I want to see similarity and img1 and img2 when the accuracy is above 77 percent.

In [33]:
def find_most_common_image_pairs_in_high_accuracy_windows(df, x_col, window_percent=0.2, 
                                                          step_percent=0.02, accuracy_threshold=0.77, 
                                                          item_col=None, top_n=20):
    """
    Find the most common img1-img2 pairs in rolling windows with high accuracy.
    """
    
    high_accuracy_trial_indices = []  # Changed to list to allow duplicates
    
    if item_col is not None and item_col in df.columns:
        items = df[item_col].unique()
        
        for item in items:
            df_item = df[df[item_col] == item].copy()
            df_item = df_item.sort_values(by=x_col).reset_index()  # Keep original index
            total_points = len(df_item)
            
            window_points = max(int(total_points * window_percent), 3)
            step_points = max(int(total_points * step_percent), 1)
            
            for j in range(0, total_points - window_points + 1, step_points):
                window = df_item.iloc[j:j+window_points]
                mean_accuracy = window['Accuracy'].mean()
                
                if mean_accuracy > accuracy_threshold:
                    # Get the original indices from the full dataframe
                    window_indices = window['index'].tolist()
                    high_accuracy_trial_indices.extend(window_indices)
    else:
        df_sorted = df.sort_values(by=x_col).reset_index()
        total_points = len(df_sorted)
        
        window_points = max(int(total_points * window_percent), 3)
        step_points = max(int(total_points * step_percent), 1)
        
        for j in range(0, total_points - window_points + 1, step_points):
            window = df_sorted.iloc[j:j+window_points]
            mean_accuracy = window['Accuracy'].mean()
            
            if mean_accuracy > accuracy_threshold:
                window_indices = window['index'].tolist()
                high_accuracy_trial_indices.extend(window_indices)
    
    # Get trials in high-accuracy windows (now with duplicates if a trial is in multiple windows)
    high_acc_trials = df.loc[high_accuracy_trial_indices]
    
    # Count each (img1, img2) pair occurrence
    image_pair_counts = high_acc_trials.groupby(['img1', 'img2']).size().reset_index(name='count')
    image_pair_counts = image_pair_counts.sort_values('count', ascending=False)
    
    # Also get the similarity values for each pair
    pair_similarity = high_acc_trials.groupby(['img1', 'img2'])[x_col].agg(['mean', 'std', 'min', 'max']).reset_index()
    
    # Merge count with similarity stats
    result = image_pair_counts.merge(pair_similarity, on=['img1', 'img2'])
    
    print(f"\n{'='*80}")
    print(f"Analysis for: {x_col}")
    print(f"{'='*80}")
    print(f"Total trial occurrences in high-accuracy windows (>{accuracy_threshold}): {len(high_acc_trials)}")
    print(f"Unique trials: {len(set(high_accuracy_trial_indices))}")
    print(f"Unique image pairs: {len(image_pair_counts)}")
    print(f"\nTop {top_n} most common image pairs:")
    print(f"{'='*80}")
    
    for idx, row in result.head(top_n).iterrows():
        print(f"{row['img1']} <-> {row['img2']}: {row['count']} occurrences "
              f"(similarity: mean={row['mean']:.3f}, std={row['std']:.3f})")
    
    return result, high_acc_trials


# Run for IT Distractor Similarity
print("\n🔵 IT DISTRACTOR SIMILARITY ANALYSIS")
it_pairs, it_trials = find_most_common_image_pairs_in_high_accuracy_windows(
    df, 
    x_col="Tested - Untested IT Distractor Similarity",
    window_percent=0.2,
    step_percent=0.02,
    accuracy_threshold=0.95,
    item_col='participant',
    top_n=20
)

# # Run for V2 Distractor Similarity
# print("\n\n🟢 V2 DISTRACTOR SIMILARITY ANALYSIS")
# v2_pairs, v2_trials = find_most_common_image_pairs_in_high_accuracy_windows(
#     df, 
#     x_col="Tested - Untested V2 Distractor Similarity",
#     window_percent=0.2,
#     step_percent=0.02,
#     accuracy_threshold=0.77,
#     item_col='participant',
#     top_n=20
# )

# Save results
it_pairs.to_csv('it_high_accuracy_image_pairs.csv', index=False)
v2_pairs.to_csv('v2_high_accuracy_image_pairs.csv', index=False)

print("\n✅ Results saved to CSV files")


🔵 IT DISTRACTOR SIMILARITY ANALYSIS

Analysis for: Tested - Untested IT Distractor Similarity
Total trial occurrences in high-accuracy windows (>0.95): 11007
Unique trials: 2847
Unique image pairs: 300

Top 20 most common image pairs:
./stimuli/105_2.jpg <-> ./stimuli/105_3.jpg: 76 occurrences (similarity: mean=-0.122, std=0.389)
./stimuli/67_2.jpg <-> ./stimuli/67_3.jpg: 71 occurrences (similarity: mean=0.108, std=0.576)
./stimuli/114_2.jpg <-> ./stimuli/114_3.jpg: 70 occurrences (similarity: mean=-0.145, std=0.500)
./stimuli/160_2.jpg <-> ./stimuli/160_3.jpg: 68 occurrences (similarity: mean=0.262, std=0.726)
./stimuli/102_2.jpg <-> ./stimuli/102_3.jpg: 66 occurrences (similarity: mean=-0.183, std=0.384)
./stimuli/63_2.jpg <-> ./stimuli/63_3.jpg: 66 occurrences (similarity: mean=0.118, std=0.555)
./stimuli/111_2.jpg <-> ./stimuli/111_3.jpg: 66 occurrences (similarity: mean=0.012, std=0.413)
./stimuli/52_2.jpg <-> ./stimuli/52_3.jpg: 66 occurrences (similarity: mean=0.155, std=0.450)

In [117]:
def find_most_common_image_pairs_in_high_accuracy_windows(df, x_col, 
                                                          window_percent=0.1,
                                                          step_percent=0.02,
                                                          accuracy_threshold=0.77,
                                                          similarity_range=None,  # NEW: (min, max) tuple
                                                          item_col=None, top_n=20):
    """
    Find the most common (img1, img2, pilot_number) combinations in rolling windows with high accuracy.
    
    Parameters:
    -----------
    similarity_range : tuple, optional
        (min, max) range to filter by similarity. E.g., (0.8, 1.1) for similarities between 0.8 and 1.1
    """
    
    high_accuracy_trial_indices = []
    window_accuracies = []
    
    if item_col is not None and item_col in df.columns:
        items = df[item_col].unique()
        
        for item in items:
            df_item = df[df[item_col] == item].copy()
            df_item = df_item.sort_values(by=x_col).reset_index()
            total_points = len(df_item)
            
            window_points = max(int(total_points * window_percent), 3)
            step_points = max(int(total_points * step_percent), 1)
            
            for j in range(0, total_points - window_points + 1, step_points):
                window = df_item.iloc[j:j+window_points]
                mean_accuracy = window['Accuracy'].mean()
                window_accuracies.append(mean_accuracy)
                
                if mean_accuracy > accuracy_threshold:
                    window_indices = window['index'].tolist()
                    high_accuracy_trial_indices.extend(window_indices)
    else:
        df_sorted = df.sort_values(by=x_col).reset_index()
        total_points = len(df_sorted)
        
        window_points = max(int(total_points * window_percent), 3)
        step_points = max(int(total_points * step_percent), 1)
        
        for j in range(0, total_points - window_points + 1, step_points):
            window = df_sorted.iloc[j:j+window_points]
            mean_accuracy = window['Accuracy'].mean()
            window_accuracies.append(mean_accuracy)
            
            if mean_accuracy > accuracy_threshold:
                window_indices = window['index'].tolist()
                high_accuracy_trial_indices.extend(window_indices)
    
    # DIAGNOSTICS
    print(f"\n{'='*80}")
    print(f"DIAGNOSTIC INFO for: {x_col}")
    print(f"{'='*80}")
    print(f"Window parameters: window_percent={window_percent}, step_percent={step_percent}")
    print(f"Threshold: {accuracy_threshold}")
    if similarity_range is not None:
        print(f"Similarity range filter: {similarity_range[0]} to {similarity_range[1]}")
    print(f"Total windows evaluated: {len(window_accuracies)}")
    print(f"Windows meeting threshold: {sum(1 for acc in window_accuracies if acc > accuracy_threshold)}")
    print(f"Max window accuracy: {max(window_accuracies) if window_accuracies else 0:.4f}")
    print(f"Min window accuracy: {min(window_accuracies) if window_accuracies else 0:.4f}")
    print(f"Mean window accuracy: {np.mean(window_accuracies) if window_accuracies else 0:.4f}")
    print(f"Trial indices collected: {len(high_accuracy_trial_indices)}")
    
    if len(high_accuracy_trial_indices) == 0:
        print(f"\n⚠️  NO trials found meeting threshold {accuracy_threshold}")
        print(f"💡 Try a lower threshold (max observed: {max(window_accuracies) if window_accuracies else 0:.4f})")
        return None, None
    
    # Get trials in high-accuracy windows
    high_acc_trials = df.loc[high_accuracy_trial_indices]
    
    # Apply similarity range filter if specified
    if similarity_range is not None:
        min_sim, max_sim = similarity_range
        high_acc_trials = high_acc_trials[
            (high_acc_trials[x_col] >= min_sim) & 
            (high_acc_trials[x_col] <= max_sim)
        ]
        print(f"After similarity filter: {len(high_acc_trials)} trials remaining")
    
    if len(high_acc_trials) == 0:
        print(f"\n⚠️  NO trials found after applying similarity filter")
        return None, None
    
    # Count each (img1, img2, pilot_number) combination
    image_pair_counts = high_acc_trials.groupby(['img1', 'img2', 'pilot_number']).size().reset_index(name='count')
    image_pair_counts = image_pair_counts.sort_values('count', ascending=False)
    
    # Get similarity stats for each combination
    pair_similarity = high_acc_trials.groupby(['img1', 'img2', 'pilot_number'])[x_col].agg([
        'mean', 'std', 'min', 'max', 'median'
    ]).reset_index()
    
    result = image_pair_counts.merge(pair_similarity, on=['img1', 'img2', 'pilot_number'])
    
    print(f"\n{'='*80}")
    print(f"RESULTS for: {x_col}")
    print(f"{'='*80}")
    print(f"Total trial occurrences: {len(high_acc_trials)}")
    print(f"Unique trials: {len(set(high_acc_trials.index))}")
    print(f"Unique image pair combinations (with pilot): {len(image_pair_counts)}")
    print(f"\nTop {top_n} most common image pair combinations:")
    print(f"{'='*80}")
    
    for idx, row in result.head(top_n).iterrows():
        print(f"Pilot {row['pilot_number']}: {row['img1']} <-> {row['img2']}")
        print(f"   Count: {row['count']} occurrences")
        print(f"   Similarity - Mean: {row['mean']:.3f}, Median: {row['median']:.3f}, "
              f"Std: {row['std']:.3f}, Range: [{row['min']:.3f}, {row['max']:.3f}]")
        print()
    
    return result, high_acc_trials


# Run for IT Distractor Similarity with similarity range filter
print("\n🔵 IT DISTRACTOR - Testing with 0.775 threshold, similarity range 0.8-1.1")
it_pairs, it_trials = find_most_common_image_pairs_in_high_accuracy_windows(
    df, 
    x_col="Tested - Untested IT Distractor Similarity",
    window_percent=0.1,
    step_percent=0.02,
    accuracy_threshold=0.775,
    similarity_range=(0.75, 0.85),  # NEW: Filter for similarities between 0.8 and 1.1
    item_col=None,
    top_n=20
)

# # Run for V2 Distractor Similarity (you can add a range here too if needed)
# print("\n\n🟢 V2 DISTRACTOR - Testing with 0.85 threshold")
# v2_pairs, v2_trials = find_most_common_image_pairs_in_high_accuracy_windows(
#     df, 
#     x_col="Tested - Untested V2 Distractor Similarity",
#     window_percent=0.1,
#     step_percent=0.02,
#     accuracy_threshold=0.85,
#     similarity_range=None,  # No filter for V2, or you can add one like (0.1, 0.15)
#     item_col=None,
#     top_n=20
# )

# Save results with pilot_number and similarity metrics included
if it_pairs is not None:
    it_pairs.to_csv('it_high_accuracy_image_pairs_filtered.csv', index=False)
    print(f"\n📊 IT pairs summary:")
    print(f"   Total unique pairs: {len(it_pairs)}")
    print(f"   Similarity range in results: [{it_pairs['min'].min():.3f}, {it_pairs['max'].max():.3f}]")
    
# if v2_pairs is not None:
#     v2_pairs.to_csv('v2_high_accuracy_image_pairs_with_pilot.csv', index=False)
#     print(f"\n📊 V2 pairs summary:")
#     print(f"   Total unique pairs: {len(v2_pairs)}")
#     print(f"   Similarity range in results: [{v2_pairs['min'].min():.3f}, {v2_pairs['max'].max():.3f}]")

print("\n✅ Results saved!")


🔵 IT DISTRACTOR - Testing with 0.775 threshold, similarity range 0.8-1.1

DIAGNOSTIC INFO for: Tested - Untested IT Distractor Similarity
Window parameters: window_percent=0.1, step_percent=0.02
Threshold: 0.775
Similarity range filter: 0.75 to 0.85
Total windows evaluated: 46
Windows meeting threshold: 1
Max window accuracy: 0.7754
Min window accuracy: 0.7102
Mean window accuracy: 0.7478
Trial indices collected: 8063
After similarity filter: 2460 trials remaining

RESULTS for: Tested - Untested IT Distractor Similarity
Total trial occurrences: 2460
Unique trials: 2460
Unique image pair combinations (with pilot): 38

Top 20 most common image pair combinations:
Pilot 1: ./stimuli/170_2.jpg <-> ./stimuli/170_3.jpg
   Count: 83 occurrences
   Similarity - Mean: 0.838, Median: 0.838, Std: 0.000, Range: [0.838, 0.838]

Pilot 1: ./stimuli/181_2.jpg <-> ./stimuli/181_3.jpg
   Count: 83 occurrences
   Similarity - Mean: 0.835, Median: 0.835, Std: 0.000, Range: [0.835, 0.835]

Pilot 1: ./stimu

pilot one is pilot5 so new stimuli
pilot two is pilot6 so old stimuli

In [118]:
def find_most_common_image_pairs_in_high_accuracy_windows(df, x_col, 
                                                          window_percent=0.1,
                                                          step_percent=0.02,
                                                          accuracy_range=(0.77, 1.0),  # NEW: (min, max)
                                                          similarity_range=None,
                                                          item_col=None, top_n=20):
    """
    Find the most common (img1, img2, pilot_number) combinations in rolling windows with specific accuracy.
    
    Parameters:
    -----------
    accuracy_range : tuple
        (min, max) range for window accuracy. E.g., (0.77, 0.85) for accuracies between 77% and 85%
    similarity_range : tuple, optional
        (min, max) range to filter by similarity. E.g., (0.8, 1.1) for similarities between 0.8 and 1.1
    """
    
    high_accuracy_trial_indices = []
    window_accuracies = []
    
    # Unpack accuracy range
    min_accuracy, max_accuracy = accuracy_range
    
    if item_col is not None and item_col in df.columns:
        items = df[item_col].unique()
        
        for item in items:
            df_item = df[df[item_col] == item].copy()
            df_item = df_item.sort_values(by=x_col).reset_index()
            total_points = len(df_item)
            
            window_points = max(int(total_points * window_percent), 3)
            step_points = max(int(total_points * step_percent), 1)
            
            for j in range(0, total_points - window_points + 1, step_points):
                window = df_item.iloc[j:j+window_points]
                mean_accuracy = window['Accuracy'].mean()
                window_accuracies.append(mean_accuracy)
                
                # Check if accuracy is within range
                if min_accuracy < mean_accuracy <= max_accuracy:
                    window_indices = window['index'].tolist()
                    high_accuracy_trial_indices.extend(window_indices)
    else:
        df_sorted = df.sort_values(by=x_col).reset_index()
        total_points = len(df_sorted)
        
        window_points = max(int(total_points * window_percent), 3)
        step_points = max(int(total_points * step_percent), 1)
        
        for j in range(0, total_points - window_points + 1, step_points):
            window = df_sorted.iloc[j:j+window_points]
            mean_accuracy = window['Accuracy'].mean()
            window_accuracies.append(mean_accuracy)
            
            # Check if accuracy is within range
            if min_accuracy < mean_accuracy <= max_accuracy:
                window_indices = window['index'].tolist()
                high_accuracy_trial_indices.extend(window_indices)
    
    # DIAGNOSTICS
    print(f"\n{'='*80}")
    print(f"DIAGNOSTIC INFO for: {x_col}")
    print(f"{'='*80}")
    print(f"Window parameters: window_percent={window_percent}, step_percent={step_percent}")
    print(f"Accuracy range: {min_accuracy} to {max_accuracy}")
    if similarity_range is not None:
        print(f"Similarity range filter: {similarity_range[0]} to {similarity_range[1]}")
    print(f"Total windows evaluated: {len(window_accuracies)}")
    print(f"Windows meeting criteria: {sum(1 for acc in window_accuracies if min_accuracy < acc <= max_accuracy)}")
    print(f"Max window accuracy: {max(window_accuracies) if window_accuracies else 0:.4f}")
    print(f"Min window accuracy: {min(window_accuracies) if window_accuracies else 0:.4f}")
    print(f"Mean window accuracy: {np.mean(window_accuracies) if window_accuracies else 0:.4f}")
    print(f"Trial indices collected: {len(high_accuracy_trial_indices)}")
    
    if len(high_accuracy_trial_indices) == 0:
        print(f"\n⚠️  NO trials found meeting criteria")
        print(f"💡 Try adjusting the accuracy range (observed range: {min(window_accuracies) if window_accuracies else 0:.4f} - {max(window_accuracies) if window_accuracies else 0:.4f})")
        return None, None
    
    # Get trials in high-accuracy windows
    high_acc_trials = df.loc[high_accuracy_trial_indices]
    
    # Apply similarity range filter if specified
    if similarity_range is not None:
        min_sim, max_sim = similarity_range
        high_acc_trials = high_acc_trials[
            (high_acc_trials[x_col] >= min_sim) & 
            (high_acc_trials[x_col] <= max_sim)
        ]
        print(f"After similarity filter: {len(high_acc_trials)} trials remaining")
    
    if len(high_acc_trials) == 0:
        print(f"\n⚠️  NO trials found after applying similarity filter")
        return None, None
    
    # Count each (img1, img2, pilot_number) combination
    image_pair_counts = high_acc_trials.groupby(['img1', 'img2', 'pilot_number']).size().reset_index(name='count')
    image_pair_counts = image_pair_counts.sort_values('count', ascending=False)
    
    # Get similarity stats for each combination
    pair_similarity = high_acc_trials.groupby(['img1', 'img2', 'pilot_number'])[x_col].agg([
        'mean', 'std', 'min', 'max', 'median'
    ]).reset_index()
    
    result = image_pair_counts.merge(pair_similarity, on=['img1', 'img2', 'pilot_number'])
    
    print(f"\n{'='*80}")
    print(f"RESULTS for: {x_col}")
    print(f"{'='*80}")
    print(f"Total trial occurrences: {len(high_acc_trials)}")
    print(f"Unique trials: {len(set(high_acc_trials.index))}")
    print(f"Unique image pair combinations (with pilot): {len(image_pair_counts)}")
    print(f"\nTop {top_n} most common image pair combinations:")
    print(f"{'='*80}")
    
    for idx, row in result.head(top_n).iterrows():
        print(f"Pilot {row['pilot_number']}: {row['img1']} <-> {row['img2']}")
        print(f"   Count: {row['count']} occurrences")
        print(f"   Similarity - Mean: {row['mean']:.3f}, Median: {row['median']:.3f}, "
              f"Std: {row['std']:.3f}, Range: [{row['min']:.3f}, {row['max']:.3f}]")
        print()
    
    return result, high_acc_trials


# # Example 1: Accuracy 77.5%-78.5%, IT similarity 0.8-1.1
# print("\n🔵 IT DISTRACTOR - Accuracy 0.775-0.785, IT similarity 0.8-1.1")
# it_pairs_1, it_trials_1 = find_most_common_image_pairs_in_high_accuracy_windows(
#     df, 
#     x_col="Tested - Untested IT Distractor Similarity",
#     window_percent=0.1,
#     step_percent=0.02,
#     accuracy_range=(0.775, 0.785),  # Max accuracy cutoff!
#     similarity_range=(0.8, 1.1),
#     item_col=None,
#     top_n=20
# )

# Example 2: Accuracy 70%-75%, IT similarity -1.0 to -0.5
# print("\n\n🔵 IT DISTRACTOR - Accuracy 0.70-0.75, IT similarity -1.0 to -0.5")
it_pairs_2, it_trials_2 = find_most_common_image_pairs_in_high_accuracy_windows(
    df, 
    x_col="Tested - Untested IT Distractor Similarity",
    window_percent=0.1,
    step_percent=0.02,
    accuracy_range=(0.74, 0.7544),  # Different accuracy range
    similarity_range=(0.9, 1.3),  # Negative similarities
    item_col=None,
    top_n=20
)






DIAGNOSTIC INFO for: Tested - Untested IT Distractor Similarity
Window parameters: window_percent=0.1, step_percent=0.02
Accuracy range: 0.74 to 0.7544
Similarity range filter: 0.9 to 1.3
Total windows evaluated: 46
Windows meeting criteria: 10
Max window accuracy: 0.7754
Min window accuracy: 0.7102
Mean window accuracy: 0.7478
Trial indices collected: 80630
After similarity filter: 8063 trials remaining

RESULTS for: Tested - Untested IT Distractor Similarity
Total trial occurrences: 8063
Unique trials: 8063
Unique image pair combinations (with pilot): 122

Top 20 most common image pair combinations:
Pilot 2: ./stimuli/94_2.jpg <-> ./stimuli/94_3.jpg
   Count: 91 occurrences
   Similarity - Mean: 1.223, Median: 1.223, Std: 0.000, Range: [1.223, 1.223]

Pilot 1: ./stimuli/233_2.jpg <-> ./stimuli/233_3.jpg
   Count: 91 occurrences
   Similarity - Mean: 1.080, Median: 1.080, Std: 0.000, Range: [1.080, 1.080]

Pilot 1: ./stimuli/221_2.jpg <-> ./stimuli/221_3.jpg
   Count: 88 occurrences
